In [1]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# from tensorflow.keras import datasets, layers, models
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
# from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
# from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras import backend as K
# import tensorflow as tf
# from tensorflow import keras
# import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback
# tf.config.optimizer.set_jit(True)
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

In [2]:
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)


In [3]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [4]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [5]:
aug = ImageDataGenerator(width_shift_range=0.1,
                height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

In [6]:
sweep_config = {

     'method': 'grid',

    'parameters': {
        'learning_rate': 
         {'values': [ 3e-2]},
        
        'reg': 
            {'values': [0.05 ,0.04, 0.01]},
        
        'decay': 
            {'values': [0.01,0.002,0.001]}
     
    
        }

    }
sweep_id = wandb.sweep(sweep_config, project= "ResNet_Cifar10_HyperParam_Adam") 

Create sweep with ID: e96a2tue
Sweep URL: https://app.wandb.ai/roastedkernel/ResNet_Cifar10_HyperParam_Adam/sweeps/e96a2tue


In [7]:
# def residual_module(data, K, stride, chanDim,
#                     reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
#     shortcut = data
    
#     bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#             momentum=bnMom)(data)
#     act1 = Activation("relu")(bn1)
    
#     conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
#                 kernel_regularizer=l2(reg))(act1)
    
    
#     bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                 momentum=bnMom)(conv1)
#     act2 = Activation("relu")(bn2)
#     conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
#                 padding="same", use_bias=False,
#     kernel_regularizer=l2(reg))(act2)
    
    
#     bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                 momentum=bnMom)(conv2)
#     act3 = Activation("relu")(bn3)
#     conv3 = Conv2D(K, (1, 1), use_bias=False,
#                 kernel_regularizer=l2(reg))(act3)
    
    
    
#     if reduce:
#         shortcut = Conv2D(K, (1, 1), strides=stride,
#             use_bias=False, kernel_regularizer=l2(reg))(act1)
    
#     x = add([conv3, shortcut])
    
    
    
#     return x






# def build_Resnet(width, height, depth, classes, stages, filters,
#                 reg=0.0001, bnEps=2e-5, bnMom=0.9):
    
    
#     inputShape = (height, width, depth)
#     chanDim = -1
#     inputs = Input(shape=inputShape)
#     x = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                             momentum=bnMom)(inputs)
#     x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)
    
    
    
#     for i in range(0, len(stages)):

#         stride = (1, 1) if i == 0 else (2, 2)
        
#         x = residual_module(x, filters[i + 1], stride,
#                 chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)
        
#         for j in range(0, stages[i] - 1):
            
#             x = residual_module(x, filters[i + 1],
#                     (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)
            
#     x = BatchNormalization(axis=chanDim, epsilon=bnEps,
#                             momentum=bnMom)(x)
#     x = Activation("relu")(x)
#     x = AveragePooling2D((8, 8))(x) 
#     x = Flatten()(x)
#     x = Dense(classes, kernel_regularizer=l2(reg))(x)
#     x = Activation("softmax", dtype='float32')(x)
    
#     model = Model(inputs, x, name="resnet")
    
    
    
#     return model


In [8]:
def train():
    
    
    
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    from tensorflow.keras import datasets, layers, models
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.callbacks import ModelCheckpoint
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, AveragePooling2D
    from tensorflow.keras.layers import ZeroPadding2D, add, Input, Activation
    from tensorflow.keras.regularizers import l2
    from sklearn.preprocessing import LabelBinarizer
    from tensorflow.keras.optimizers import SGD
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.datasets import cifar10
    from tensorflow.keras import backend as K
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    
    
    
    
    
    def residual_module(data, K, stride, chanDim,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9, reduce=False):
    
        shortcut = data

        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                momentum=bnMom)(data)
        act1 = Activation("elu")(bn1)

        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
                    kernel_regularizer=l2(reg))(act1)


        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                    momentum=bnMom)(conv1)
        act2 = Activation("elu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
                    padding="same", use_bias=False,
        kernel_regularizer=l2(reg))(act2)


        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
                    momentum=bnMom)(conv2)
        act3 = Activation("elu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
                    kernel_regularizer=l2(reg))(act3)



        if reduce:
            shortcut = Conv2D(K, (1, 1), strides=stride,
                use_bias=False, kernel_regularizer=l2(reg))(act1)

        x = add([conv3, shortcut])
    
    
    
        return x






    def build_Resnet(width, height, depth, classes, stages, filters,
                    reg=0.0001, bnEps=2e-5, bnMom=0.9):


        inputShape = (height, width, depth)
        chanDim = -1
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                                momentum=bnMom)(inputs)
        x = Conv2D(filters[0], (3, 3), use_bias=False,padding="same", kernel_regularizer=l2(reg))(x)



        for i in range(0, len(stages)):

            stride = (1, 1) if i == 0 else (2, 2)

            x = residual_module(x, filters[i + 1], stride,
                    chanDim, reduce=True, bnEps=bnEps, bnMom=bnMom)

            for j in range(0, stages[i] - 1):

                x = residual_module(x, filters[i + 1],
                        (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
                                momentum=bnMom)(x)
        x = Activation("elu")(x)
        x = AveragePooling2D((8, 8))(x) 
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax", dtype='float32')(x)

        model = Model(inputs, x, name="resnet")



        return model

    
    wandb.init()
    configs = {'learning_rate': 2e-2,'reg': 0.0009,'decay':0.001 }
    config = wandb.config
    config.epochs = 100
    
    
    
    model = build_Resnet(32, 32, 3, 10, (9, 9, 9),
                                (64, 64, 128, 256), reg=wandb.config.reg)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

    metric = "val_accuracy"

    checkpoint = ModelCheckpoint("ResNet_Cifar10.h5", monitor= metric, verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto')
    
    
    #n_epochs = 100
    batch = 128
#     s = 100 * len(trainX) // batch # number of steps in 20 epochs (batch size = 32)
#     learning_rate0 = keras.optimizers.schedules.ExponentialDecay(wandb.config.learning_rate, s
#                                                                  , wandb.config.decay)

    opt = Adam(wandb.config.learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07
               , decay=wandb.config.decay, amsgrad=True)
    #opt = opt = SGD(lr= wandb.config.learning_rate ,decay = wandb.config.decay ,  momentum= 0.9, nesterov=True)
    
    
    model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=['accuracy'])
    
    wandb_callback = WandbCallback(data_type= "image", validation_data=(testX, testY)
                                 , labels = labelNames)

    history = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch), epochs=config.epochs, 
                    validation_data=(testX, testY),steps_per_epoch=len(trainX) // batch,
                    callbacks=[es,checkpoint,wandb_callback], verbose=10)


In [9]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: inaavqxa with config:
	decay: 0.01
	learning_rate: 0.03
	reg: 0.05
wandb: Agent Started Run: inaavqxa


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.34270, saving model to ResNet_Cifar10.h5
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.34270 to 0.40750, saving model to ResNet_Cifar10.h5
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.40750 to 0.50720, saving model to ResNet_Cifar10.h5
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.50720 to 0.60360, saving model to ResNet_Cifar10.h5
Epoch 5/100


wandb: Network error resolved after 0:00:34.751356, resuming normal operation.
wandb: Network error resolved after 0:00:39.535140, resuming normal operation.



Epoch 00005: val_accuracy improved from 0.60360 to 0.60430, saving model to ResNet_Cifar10.h5
Epoch 6/100

Epoch 00006: val_accuracy improved from 0.60430 to 0.62240, saving model to ResNet_Cifar10.h5
Epoch 7/100

Epoch 00007: val_accuracy improved from 0.62240 to 0.65900, saving model to ResNet_Cifar10.h5
Epoch 8/100

Epoch 00008: val_accuracy improved from 0.65900 to 0.68170, saving model to ResNet_Cifar10.h5
Epoch 9/100


wandb: Network error resolved after 0:00:32.799173, resuming normal operation.


...

wandb: Wandb version 0.10.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Unable to parse config file; probably being modified by user process?


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100


Process Process-5:
Traceback (most recent call last):
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/wandb/wandb_agent.py", line 64, in _start
    function()
  File "<ipython-input-8-7d67ea591195>", line 149, in train
    history = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch), epochs=config.epochs,
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1465, in fit_generator
    return self.fit(
  File "/home/moazmohamed/mi

wandb: Network error resolved after 1:27:19.628654, resuming normal operation.
Retry attempt failed:
Traceback (most recent call last):
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/moazmohamed/miniconda3/envs/ML_CV/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response

wandb: Network error (ConnectionError), entering retry loop. See /home/moazmohamed/Desktop/data/uni_subjects/uni_Machine_Learning/Deep_Learning_Fundamentals/assign_2/wandb/debug.log for full traceback.
wandb: Network error resolved after 0:00:53.993399, resuming normal operation.
wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.
